#### benodige libraries importeren

In [1]:
import pandas as pd
import numpy as np

#libraries om xml mee te verwerken
import xml.etree.ElementTree
from lxml import etree

#library om alle bestanden in een map te kunnen lezen
import glob

#libraries om te kunnen voorspellen
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression

#library om dataset in train en testset te splitsen
from sklearn.model_selection import train_test_split

#bokeh libraries voor plotten van grafieken
from bokeh.io import output_notebook, hplot, show
from bokeh.layouts import row, gridplot, widgetbox
from bokeh.charts import Histogram, BoxPlot, Scatter, Line, Bar
from bokeh.plotting import figure, ColumnDataSource, show
from bokeh.models import CustomJS, Slider, Span

output_notebook()

Loading BokehJS ...

# Data munging

#### Data uit het gps txt bestand lezen en in een dataframe zetten

In [2]:
spelerdata = pd.read_csv('gps/20160410 FC Utrecht - NEC/b6a6aeaf67e640fe9eb72d888ba046f9.txt',skiprows=3, header=None)
spelerdata.columns = ['GPStijd', 'dt(ms)', 'lat', 'long', 'snelheid(m/s)', 'acc_x(g-force)', 'acc_y', 'acc_z', 'gyro_x(deg/s)', 'gyro_y',
'gyro_z', 'NOTOFINTEREST']
spelerdata.head()

,GPStijd,dt(ms),lat,long,snelheid(m/s),acc_x(g-force),acc_y,acc_z,gyro_x(deg/s),gyro_y,gyro_z,NOTOFINTEREST
0,-,8,NaN,NaN,NaN,0.000,0.000,0.000,0.000,0.000,0.000,0
1,-,8,NaN,NaN,NaN,0.309,0.684,0.277,-33.756,-4.031,14.275,0
2,-,8,NaN,NaN,NaN,0.452,0.695,0.301,-77.206,33.130,-28.519,0
3,-,8,NaN,NaN,NaN,0.482,0.576,0.234,-166.366,25.267,-28.702,0
4,-,8,NaN,NaN,NaN,0.468,0.650,0.515,-150.873,-0.544,47.177,0


#### We halen hier de starttijd op van de wedstrijd uit het xml bestand

In [3]:
root = xml.etree.ElementTree.parse('new-xml/20160410 FC Utrecht - NEC-new.xml').getroot()
starttime = root.getchildren()[0].getchildren()[0].text
starttime = pd.to_datetime(starttime)
#starttime = float(str(starttime.hour) + str(starttime.minute) + str(starttime.second))
print('De wedstrijd begon om: ' + str(starttime))

De wedstrijd begon om: 2016-04-10 12:24:38.930000


#### We verwijderen de rijen zonder GPStijd uit de spelerdata (1)

In [4]:
newdata = spelerdata.loc[spelerdata['GPStijd'] != '-']
newdata['GPStijd'] = newdata['GPStijd'].astype(np.float)
newdata = newdata.reset_index()
del newdata['index']
newdata.head()

C:\Users\lucas\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,GPStijd,dt(ms),lat,long,snelheid(m/s),acc_x(g-force),acc_y,acc_z,gyro_x(deg/s),gyro_y,gyro_z,NOTOFINTEREST
0,115451.3,8,52.078556,5.145593,1.98,0.910,-0.830,0.319,-27.206,31.863,-11.145,0
1,115451.5,8,52.078553,5.145591,2.08,0.004,0.037,-0.219,-90.519,48.702,-49.710,0
2,115451.6,8,52.078550,5.145591,1.97,0.360,-0.091,-0.037,-133.304,81.634,-60.740,0
3,115451.7,8,52.078549,5.145590,1.71,1.579,-0.875,-0.350,-40.718,-41.573,-79.221,0
4,115451.8,8,52.078548,5.145590,1.78,1.324,-1.498,0.430,33.985,-58.855,-51.282,0


#### We voegen een nieuwe kollom toe aan de spelerdata met de GPStijd als timestamp (2)

In [5]:
year = str(starttime.year)
month = str(starttime.month)
day = str(starttime.day)
timestamps = []
for i in range(0, len(newdata)):
    gpstijd = newdata.loc[i]['GPStijd']
    uur = (np.floor(gpstijd/10000)).astype(int)
    minuut = (np.floor((gpstijd - uur * 10000)/100)).astype(int)
    seconde = ((gpstijd - uur * 10000) - minuut * 100)
    tijd = str(year + "-" + month + "-" + day + " " + str(uur) + ":"+ str(minuut) + ":" + str(seconde))
    timestamps.append(pd.to_datetime(tijd))
newdata['GPStimestamp'] = timestamps
newdata.head()

,GPStijd,dt(ms),lat,long,snelheid(m/s),acc_x(g-force),acc_y,acc_z,gyro_x(deg/s),gyro_y,gyro_z,NOTOFINTEREST,GPStimestamp
0,115451.3,8,52.078556,5.145593,1.98,0.910,-0.830,0.319,-27.206,31.863,-11.145,0,2016-04-10 11:54:51.300
1,115451.5,8,52.078553,5.145591,2.08,0.004,0.037,-0.219,-90.519,48.702,-49.710,0,2016-04-10 11:54:51.500
2,115451.6,8,52.078550,5.145591,1.97,0.360,-0.091,-0.037,-133.304,81.634,-60.740,0,2016-04-10 11:54:51.600
3,115451.7,8,52.078549,5.145590,1.71,1.579,-0.875,-0.350,-40.718,-41.573,-79.221,0,2016-04-10 11:54:51.700
4,115451.8,8,52.078548,5.145590,1.78,1.324,-1.498,0.430,33.985,-58.855,-51.282,0,2016-04-10 11:54:51.800


#### De rijen verwijderen die voor de starttijd van de wedstrijd zijn

In [6]:
newdata = newdata[newdata['GPStimestamp'] >= starttime]
newdata = newdata.reset_index()
del newdata['index']
newdata

,GPStijd,dt(ms),lat,long,snelheid(m/s),acc_x(g-force),acc_y,acc_z,gyro_x(deg/s),gyro_y,gyro_z,NOTOFINTEREST,GPStimestamp
0,122439.0,8,52.079382,5.144358,8.71,0.402,-0.774,0.140,21.313,-43.084,35.893,0,2016-04-10 12:24:39.000
1,122439.1,9,52.079386,5.144348,8.63,0.534,-0.931,0.067,-24.092,9.115,-7.969,0,2016-04-10 12:24:39.100
2,122439.2,8,52.079389,5.144338,8.54,0.573,-0.879,0.138,4.947,-17.695,9.878,0,2016-04-10 12:24:39.200
3,122439.3,8,52.079393,5.144329,8.46,0.531,-1.107,0.037,-23.038,21.710,-9.924,0,2016-04-10 12:24:39.300
4,122439.4,8,52.079397,5.144320,8.40,0.396,-0.990,0.021,-21.237,-7.969,5.038,0,2016-04-10 12:24:39.400
5,122439.5,9,52.079401,5.144312,8.33,0.576,-0.859,0.020,17.786,-62.550,-20.382,0,2016-04-10 12:24:39.500
6,122439.6,8,52.079404,5.144304,8.25,0.272,-0.895,-0.080,-4.809,-33.176,33.527,0,2016-04-10 12:24:39.600
7,122439.7,8,52.079407,5.144297,8.17,0.756,-0.726,0.048,-11.359,-7.954,-5.924,0,2016-04-10 12:24:39.700
8,122439.8,8,52.079410,5.144289,8.07,0.728,-0.650,-0.026,18.153,-0.855,3.969,0,2016-04-10 12:24:39.800
9,122439.9,9,52.079412,5.144281,7.97,0.491,-0.744,0.142,22.519,-13.954,0.595,0,2016-04-10 12:24:39.900


#### We stoppen alle codes, begin en eindtijd in seconden van een event van alle wedstrijden in een dataframe (3)

In [7]:
xmlfiles = glob.glob("new-xml/*.xml")
codesall  = []
#loop door de verschillende xmlfiles
for xmlfile in xmlfiles:
    wedstrijd = xmlfile[16:-8]
    root = xml.etree.ElementTree.parse(xmlfile).getroot()
    beginwedstrijd = pd.to_datetime("")
    #loop door de elementen van de xmlfile
    for element in root.getchildren():
        for childelement in element:
            if(childelement.tag) == 'start_time':
                beginwedstrijd = pd.to_datetime(childelement.text)
            if childelement.tag == 'instance':
                eencode = []
                #als het element een instance (event) is, loop door zijn children heen
                for childelement2 in childelement:
                    tag = childelement2.tag
                    if tag == 'start' or tag == 'end' or tag == 'code':
                        eencode.append(childelement2.text)
                        if tag == 'code':
                            break;
                eencode.append(beginwedstrijd)
                eencode.append(wedstrijd)
                #voeg de informatie van één code toe aan de lijst met alle codes
                codesall.append(eencode)
codesall = pd.DataFrame(codesall)
#zet de kollomnamen
codesall.columns = ['start (sec)', 'eind (sec)' , 'code', 'beginwedstrijd', 'wedstrijd']
#zet de kollomen om naar type float
codesall['start (sec)'] = codesall['start (sec)'].astype(float)
codesall['eind (sec)'] = codesall['eind (sec)'].astype(float)
codesall

,start (sec),eind (sec),code,beginwedstrijd,wedstrijd
0,4.311043,40.971188,TEG,2016-04-10 12:24:38.930,FC Utrecht - NEC
1,6.215018,15.134795,M2 TEG,2016-04-10 12:24:38.930,FC Utrecht - NEC
2,6.311043,40.971188,Start Jaimy,2016-04-10 12:24:38.930,FC Utrecht - NEC
3,6.311043,40.971188,Verdedigen,2016-04-10 12:24:38.930,FC Utrecht - NEC
4,8.220000,14.040000,AFTRAP WB,2016-04-10 12:24:38.930,FC Utrecht - NEC
5,12.134795,21.975015,L2 TEG,2016-04-10 12:24:38.930,FC Utrecht - NEC
6,18.975015,26.262914,M1 TEG,2016-04-10 12:24:38.930,FC Utrecht - NEC
7,20.447134,40.971188,VERD. Compact NEC,2016-04-10 12:24:38.930,FC Utrecht - NEC
8,20.447134,40.971188,VERD. Druk NEC,2016-04-10 12:24:38.930,FC Utrecht - NEC
9,23.262914,30.963214,R1 TEG,2016-04-10 12:24:38.930,FC Utrecht - NEC


#### We halen alle rijen waarvan de code een vak (L1, M2, R3) van NEC is uit de lijst met alle codes

In [141]:
tempcodes = codesall[(codesall['code'].str.len() < 20) & (codesall['code'].str.len() == 6) & (codesall['code'].str.contains('NEC'))].sort_values('code').reset_index()
del tempcodes['index']
tempcodes

,start (sec),eind (sec),code,beginwedstrijd,wedstrijd
0,1856.423369,1862.255245,L1 NEC,NaT,NEC - Cambuur
1,1374.117652,1380.221668,L1 NEC,NaT,NEC - Cambuur
2,837.862121,842.139642,L1 NEC,NaT,NEC - Roda JC
3,2212.194415,2219.506642,L1 NEC,2016-04-20 16:01:49.970,PEC - NEC
4,2504.794280,2514.466713,L1 NEC,2016-04-20 16:01:49.970,PEC - NEC
5,5359.876106,5364.084404,L1 NEC,NaT,NEC - Cambuur
6,3174.490468,3181.648148,L1 NEC,NaT,NEC - Roda JC
7,1508.106520,1526.090247,L1 NEC,2016-04-20 16:01:49.970,PEC - NEC
8,3731.884326,3739.595956,L1 NEC,2016-04-20 16:01:49.970,PEC - NEC
9,1972.197869,1985.355640,L1 NEC,NaT,NEC - Roda JC


#### We delen de vakken in op een lengte en een breedte vak (4)
breedte: Links wordt 1, Midden wordt 2, Links wordt 3
<br/>
lengte: is al 1,2,3,4

In [9]:
lengtecolumn = []
for i in range(0, len(tempcodes)):
    rij = tempcodes.loc[i]
    lengtecolumn.append(int(rij['code'][1:-4]))
breedtecolumn = []
for i in range(0, len(tempcodes)):
    rij = tempcodes.loc[i]
    breedtechar = rij['code'][0:1]
    if breedtechar == 'L':
        breedtecolumn.append(1)
    elif breedtechar == 'M':
        breedtecolumn.append(2)
    elif breedtechar == 'R':
        breedtecolumn.append(3)    
tempcodes['breedtevak'] = breedtecolumn
tempcodes['lengtevak'] = lengtecolumn
tempcodes

,start (sec),eind (sec),code,beginwedstrijd,wedstrijd,breedtevak,lengtevak
0,1856.423369,1862.255245,L1 NEC,NaT,NEC - Cambuur,1,1
1,1374.117652,1380.221668,L1 NEC,NaT,NEC - Cambuur,1,1
2,837.862121,842.139642,L1 NEC,NaT,NEC - Roda JC,1,1
3,2212.194415,2219.506642,L1 NEC,2016-04-20 16:01:49.970,PEC - NEC,1,1
4,2504.794280,2514.466713,L1 NEC,2016-04-20 16:01:49.970,PEC - NEC,1,1
5,5359.876106,5364.084404,L1 NEC,NaT,NEC - Cambuur,1,1
6,3174.490468,3181.648148,L1 NEC,NaT,NEC - Roda JC,1,1
7,1508.106520,1526.090247,L1 NEC,2016-04-20 16:01:49.970,PEC - NEC,1,1
8,3731.884326,3739.595956,L1 NEC,2016-04-20 16:01:49.970,PEC - NEC,1,1
9,1972.197869,1985.355640,L1 NEC,NaT,NEC - Roda JC,1,1


#### Alle goals van NEC worden in een apparte dataframe gezet

In [10]:
goalsnec = codesall[codesall['code'] == 'Goal NEC'].reset_index()
del goalsnec['index']
goalsnec

,start (sec),eind (sec),code,beginwedstrijd,wedstrijd
0,2589.883302,2609.883302,Goal NEC,2016-04-10 12:24:38.930,FC Utrecht - NEC
1,3243.692465,3263.692465,Goal NEC,NaT,NEC - Cambuur
2,4188.847565,4208.847565,Goal NEC,NaT,NEC - Cambuur
3,1399.655065,1419.655065,Goal NEC,NaT,NEC - Roda JC


#### we voegen aan beide dataframes een timestamp kollom toe, zodat we hier later mee kunnen vergelijken

In [11]:
goalsnec['timestamps'] = pd.to_datetime(goalsnec['start (sec)'] * 1000000000)
tempcodes['timestamps'] = pd.to_datetime(tempcodes['start (sec)'] * 1000000000)

#### Er word per goal vergeleken welke timestamp van de tempcodes dataframe er het dichste bij ligt.

In [12]:
closest = {}
for i in range(0, len(tempcodes)):
    timestamp = tempcodes.loc[i]['timestamps']
    wedstrijd = tempcodes.loc[i]['wedstrijd']
    for i in range(0, len(goalsnec)):
        if wedstrijd == goalsnec.loc[i]['wedstrijd']:
            if wedstrijd not in closest:
                closest[wedstrijd] = {}
            if goalsnec.loc[i]['timestamps'] not in closest[wedstrijd]:
                closest[wedstrijd][goalsnec.loc[i]['timestamps']] = timestamp
            for key in closest[wedstrijd]:
                prevclosesttime = closest[wedstrijd][goalsnec.loc[i]['timestamps']]
                if abs((goalsnec.loc[i]['timestamps'] - timestamp).total_seconds()) < abs((goalsnec.loc[i]['timestamps'] - prevclosesttime).total_seconds()):
                    closest[wedstrijd][goalsnec.loc[i]['timestamps']] = timestamp
closest    

{' FC Utrecht - NEC': {Timestamp('1970-01-01 00:43:09.883301861'): Timestamp('1970-01-01 00:43:05.370434190')},
 ' NEC - Cambuur': {Timestamp('1970-01-01 00:54:03.692465097'): Timestamp('1970-01-01 00:54:04.255785547'),
  Timestamp('1970-01-01 01:09:48.847564678'): Timestamp('1970-01-01 01:09:50.464046433')},
 ' NEC - Roda JC': {Timestamp('1970-01-01 00:23:19.655065267'): Timestamp('1970-01-01 00:23:19.850208898')}}

#### Stop van alle rijen die voorkwamen in de vorige dictionary het id in een lijst

In [13]:
indexlijst = []
for key in closest:
    value = closest[key]
    for keyvalue in value:
        indexlijst.append(tempcodes[tempcodes['timestamps'] == value[keyvalue]].index[0])

#### Voeg een kollom toe waarin staat of er een goal is gemaakt vanuit dit vak of niet (0 = nee, 1 = ja) (5)

In [14]:
goals = []
for i in range(0, len(tempcodes)):
    if i in indexlijst:
        goals.append(1)
    else:
        goals.append(0)
tempcodes['goal'] = goals
tempcodes.head()

,start (sec),eind (sec),code,beginwedstrijd,wedstrijd,breedtevak,lengtevak,timestamps,goal
0,1856.423369,1862.255245,L1 NEC,NaT,NEC - Cambuur,1,1,1970-01-01 00:30:56.423369114,0
1,1374.117652,1380.221668,L1 NEC,NaT,NEC - Cambuur,1,1,1970-01-01 00:22:54.117651651,0
2,837.862121,842.139642,L1 NEC,NaT,NEC - Roda JC,1,1,1970-01-01 00:13:57.862121441,0
3,2212.194415,2219.506642,L1 NEC,2016-04-20 16:01:49.970,PEC - NEC,1,1,1970-01-01 00:36:52.194415339,0
4,2504.794280,2514.466713,L1 NEC,2016-04-20 16:01:49.970,PEC - NEC,1,1,1970-01-01 00:41:44.794279630,0


# Statistische analyse, hoeveel afstand heeft een speler afgelegd? (6)

In [15]:
afstand = 0
vorigetijd = starttime
for i in range(0, len(newdata)):
    snelheid = newdata.loc[i]['snelheid(m/s)']
    tijdverschil = (newdata.loc[i]['GPStimestamp'] - vorigetijd).total_seconds()
    vorigetijd = newdata.loc[i]['GPStimestamp']
    if(tijdverschil < 1):
        afstand += tijdverschil * snelheid
print("Gelopen afstand in meters: " + str(afstand))

Gelopen afstand in meters: 9454.4107


#### Bereken wanneer de pauze is: als er meer dan vijf minuten geen data is, is dit waarschijnlijk de pauze (7)

In [16]:
vorigetijd = starttime
for i in range(0, len(newdata)):
    rij = newdata.loc[i]
    if((rij['GPStimestamp'] - vorigetijd).total_seconds() > 300):
        print('pauze om ' + str(rij['GPStimestamp']) + ', lengte pauze: ' + str(rij['GPStimestamp'] - vorigetijd))
    vorigetijd = rij['GPStimestamp']

pauze om 2016-04-10 13:26:42.100000, lengte pauze: 0 days 00:06:20


Maar zes minuten pauze?
Waarschijnlijk is de gps tracker niet gelijk uitgezet toen de pauze begon, of aangezet voordat de pauze is afgelopen

# Machine learning

### Met deze voorspelling kun je een vak aangeven. Er word dan voorspeld of je wel (1) of niet (0) scored vanuit dit vak (8)

In [17]:
x = tempcodes[['lengtevak','breedtevak']].values.tolist()
y = tempcodes['goal'].values.tolist()
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(x, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

#### Voorspel of NEC scored vanuit vak Lente: 4, Breedte: 1 (Links)

In [18]:
neigh.predict([[4,1]])

array([0])

Er is voorspeld dat NEC niet zal scoren vanuit dit vak

#### De score van onze berekening (in hoeveel procent van de gevallen heeft hij het goed?)

In [19]:
neigh.score(tempcodes[['lengtevak','breedtevak']].values.tolist(),  tempcodes['goal']) * 100

99.731543624161077

## Voorspel wat de kans is dat er gescored word vanuit een bepaald vak
#### Zet alle goals van NEC in een apparte dataframe

In [20]:
goalsnec = tempcodes[tempcodes['goal'] == 1].reset_index()
del goalsnec['index']
goalsnec

,start (sec),eind (sec),code,beginwedstrijd,wedstrijd,breedtevak,lengtevak,timestamps,goal
0,3244.255786,3249.935761,L4 NEC,NaT,NEC - Cambuur,1,4,1970-01-01 00:54:04.255785547,1
1,1399.850209,1412.385762,L4 NEC,NaT,NEC - Roda JC,1,4,1970-01-01 00:23:19.850208898,1
2,4190.464046,4195.384048,L4 NEC,NaT,NEC - Cambuur,1,4,1970-01-01 01:09:50.464046433,1
3,2585.370434,2600.474685,R4 NEC,2016-04-10 12:24:38.930,FC Utrecht - NEC,3,4,1970-01-01 00:43:05.370434190,1


#### Tel hoeveel goals er in totaal zijn gemaakt en hoeveel vanuit welk vak

In [21]:
goals = {}
totaal = 0
for i in range(0, len(goalsnec)):
    code = goalsnec.loc[i]['code']
    if code in goals:
        goals[code] += 1
    else:
        goals[code] = 1
    totaal += 1

#### bereken hoeveel procent van de goals is gemaakt uit een vak (9)

In [22]:
for key in goals:
    print(str(goals[key]/totaal * 100) + "% van de goals zijn gemaakt vanuit vak " + key)

75.0% van de goals zijn gemaakt vanuit vak L4 NEC
25.0% van de goals zijn gemaakt vanuit vak R4 NEC


## Voorspel hoe lange afstand een speler heeft afgelegd om een bepaalde tijd in de wedstrijd

#### Voeg gelopen afstand kollom toe aan dataset

In [23]:
afstand = 0
afstanden = []
vorigetijd = starttime
for i in range(0, len(newdata)):
    rij = newdata.loc[i]
    snelheid = rij['snelheid(m/s)']
    tijdverschil = (rij['GPStimestamp'] - vorigetijd).total_seconds()
    vorigetijd = rij['GPStimestamp']
    if(tijdverschil < 1):
        afstand += tijdverschil * snelheid
    afstanden.append(afstand)
newdata['gelopen afstand (m)'] = afstanden
newdata.head()

,GPStijd,dt(ms),lat,long,snelheid(m/s),acc_x(g-force),acc_y,acc_z,gyro_x(deg/s),gyro_y,gyro_z,NOTOFINTEREST,GPStimestamp,gelopen afstand (m)
0,122439.0,8,52.079382,5.144358,8.71,0.402,-0.774,0.140,21.313,-43.084,35.893,0,2016-04-10 12:24:39.000,0.6097
1,122439.1,9,52.079386,5.144348,8.63,0.534,-0.931,0.067,-24.092,9.115,-7.969,0,2016-04-10 12:24:39.100,1.4727
2,122439.2,8,52.079389,5.144338,8.54,0.573,-0.879,0.138,4.947,-17.695,9.878,0,2016-04-10 12:24:39.200,2.3267
3,122439.3,8,52.079393,5.144329,8.46,0.531,-1.107,0.037,-23.038,21.710,-9.924,0,2016-04-10 12:24:39.300,3.1727
4,122439.4,8,52.079397,5.144320,8.40,0.396,-0.990,0.021,-21.237,-7.969,5.038,0,2016-04-10 12:24:39.400,4.0127


#### Voeg kollom toe aan dataset waarin staat hoeveel seconden na de start van de wedstrijd de rij is

In [24]:
startseccollumn = []
for i in range(0, len(newdata)):
    startseccollumn.append((newdata.loc[i]['GPStimestamp'] - starttime).total_seconds())
newdata['start (sec)'] = startseccollumn
newdata.head()

,GPStijd,dt(ms),lat,long,snelheid(m/s),acc_x(g-force),acc_y,acc_z,gyro_x(deg/s),gyro_y,gyro_z,NOTOFINTEREST,GPStimestamp,gelopen afstand (m),start (sec)
0,122439.0,8,52.079382,5.144358,8.71,0.402,-0.774,0.140,21.313,-43.084,35.893,0,2016-04-10 12:24:39.000,0.6097,0.07
1,122439.1,9,52.079386,5.144348,8.63,0.534,-0.931,0.067,-24.092,9.115,-7.969,0,2016-04-10 12:24:39.100,1.4727,0.17
2,122439.2,8,52.079389,5.144338,8.54,0.573,-0.879,0.138,4.947,-17.695,9.878,0,2016-04-10 12:24:39.200,2.3267,0.27
3,122439.3,8,52.079393,5.144329,8.46,0.531,-1.107,0.037,-23.038,21.710,-9.924,0,2016-04-10 12:24:39.300,3.1727,0.37
4,122439.4,8,52.079397,5.144320,8.40,0.396,-0.990,0.021,-21.237,-7.969,5.038,0,2016-04-10 12:24:39.400,4.0127,0.47


#### verdeel de dataset in een training en test set

In [25]:
X_train, X_test, y_train, y_test = train_test_split(newdata[['start (sec)']], newdata[['gelopen afstand (m)']])

#### train de formule

In [26]:
linreg = LinearRegression(fit_intercept=False)
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

#### Voorspel hoeveel meter de speler heeft gelopen na 2000 seconden (10)

In [27]:
aantalsec = 2000
print(str(aantalsec) + " seconden nadat de wedstrijd is begonnen heeft deze speler waarschijnlijk " + str(linreg.predict(aantalsec)[0][0]) + " meter afgelegd")

2000 seconden nadat de wedstrijd is begonnen heeft deze speler waarschijnlijk 2750.0031043 meter afgelegd


In [28]:
print("De variantie score van de lineare regressie is: " + str(round(linreg.score(X_test, y_test),2)))

De variantie score van de lineare regressie is: 0.99


# Visualisatie

#### We zetten het aantal goals vanuit een vak in een dataframe

In [137]:
goals
goalslist = []
for key in goals:
    goalslist.append([key, goals[key]])
goaldf = pd.DataFrame(goalslist)
goaldf.columns = ['vak', 'aantal goals']
goaldf

,vak,aantal goals
0,L4 NEC,3
1,R4 NEC,1


#### plot het precentage van de goals vanuit een vak (11)

In [139]:
barplot = figure(title="Precentages",
            x_range=goaldf['vak'].unique().tolist(), 
             y_range=[0,100])
for i in range(0, len(goaldf)):
    procent = goaldf.loc[i]['aantal goals']/totaal * 100
    barplot.rect(y=0, x=i+1, width=0.5, height=procent*2)
barplot.xaxis.axis_label = "Vak"
barplot.yaxis.axis_label = "Percentage"
show(barplot)

#### De rode lijn is hoeveel meter de speler om een bepaalde tijd heeft afgelegd, de blauwe lijn is onze voorspelling (11)

In [140]:
linregfig = figure()
linregfig.scatter(newdata['start (sec)'], newdata['gelopen afstand (m)'], color="red")
prediction = pd.DataFrame(linreg.predict(X_train))
linregfig.line(X_train.values.ravel(), prediction.values.ravel(), color="blue")
linregfig.yaxis.axis_label = "Tijd (sec)"
linregfig.xaxis.axis_label = "Gelopen afstand (meter)"
show(linregfig)

#### In de categorical dot plot kun je zien rond welke tijd NEC balbezit heeft in tijdsduur. De grootte van elke bolletje geeft aan hoelang de bal zich in een vak bevind. De dashed streep geeft aan dat NEC heeft gescored. Aan de kleur van de balletjes en de streep is te zien om welke wedstrijd het gaat. Wanneer een streep over een bolletje met dezelfde kleur gaat is het waarschijnlijk dat er vanuit dit vak is gescored.

In [33]:
dot = figure(title="Categorical Dot Plot",
            y_range=tempcodes['code'].unique().tolist(), 
             x_range=[0,int(tempcodes['start (sec)'].max() + 1)])
colors = {' NEC - Cambuur': 'green' , ' NEC - Roda JC' : 'blue',' PEC - NEC' : 'red',' FC Utrecht - NEC' : 'yellow'}
for i in tempcodes['wedstrijd'].unique():
    icodes = tempcodes[tempcodes['wedstrijd'] == i]
    y = icodes['code'].tolist()
    x =  icodes['start (sec)'].values.tolist()
    dot.circle(x=x, y=y, size=icodes['eind (sec)'] - icodes['start (sec)'], legend=i, fill_color=colors[i], color=colors[i])
for i in range(0, len(goalsnec)):
    goal = goalsnec.loc[i]
    goal = Span(location=goal['start (sec)'], dimension='height', line_color=colors[goal['wedstrijd']], line_dash='dashed', line_width=3)
    dot.add_layout(goal)
dot.legend.location = "bottom_right"
show(dot)